# Introduction #

In the next few lessons, we'll use feature engineering to capture the temporal relationships we discussed in Lesson 1. In terms of feature engineering, features derived from:
- the *time index* express *time dependence*
- the *past series values* express *serial dependence*

As time dependent properties, trend and seasonality we'll be able to model by creating encodings of the time index considered as a kind of categorical variable. Serial dependence we can model through *lag embeddings*, features created from past values of a time series.

We'll pull these features together using `scikit-learn`'s `LinearRegression` to get an additive model somewhat like Prophet's:

```
target = trend + seasonality + lag features + remainder
```

Our choice of linear regression is mostly for simplicity and convenience. You could (with some limitations we'll discuss later) use these same features to make a forecaster out of almost any other machine learning model, like XGBoost or neural nets.

# Trend #

In this lesson, we'll learn about trend. The **trend** of a time series is the long-term change in the mean of the series. A trend will often be driven by some kind of regular, persistant accumulation, like the <note>TODO: example</note>.

For a change to be part of the trend, it should occur over a longer period than any seasonal changes. For example, if a time series had a weekly seasonality, any trend would have to be a change over a period longer than seven days.

We will require in addition that the trend be predictable purely from the time the observation was taken, that it should be a *time dependent* property of the series. This is to distinguish a trend (which should be predictable) from a merely random accumulation (which is not predictable).

<img>Trend vs. random walk</img>

<note>TODO: explain figure</note>

A trend we should be able to extrapolate. Trying to extrapolate some kind of trend for the series on the right would not be successful, since future values are just as likely to go up as they are to go down.

It's important to understand the distinction between a true trend and merely a persistent random accumulation because many time series (especially financial time series) that we would most like to forecast do appear to behave more or less as random walks, and we would be misled in attempting to forecast a trend. For a given time series, domain research can help you determine whether a trend might actually exist -- it may not be obvious from the series itself.

# Moving Average Plots #

We can make the trend in a series more apparent by averaging over the seasonal periods with a **moving average plot**. In a moving average plot, we compute the average of the values within a *sliding window*. Each point on the graph represents the average of all the points that fall within the window on either side.

<figure style="padding: 1em;">
<img src="https://i.imgur.com/EZOXiPs.gif" width=400, alt="An animated plot showing an undulating curve slowly increasing with a moving average line developing from left to right within a window of 12 points (in red).">
<figcaption style="textalign: center; font-style: italic"><center>
</center></figcaption>
</figure>

To help isolate the trend, choose a window size at least as long as the longest seasonal period; this will help smooth over any seasonal effects. The series above has monthly observations and a yearly seasonality, so we chose a moving window of size 12.

<note>TODO: explain animation more</note>

When used for visualization, taking the average at the center of the window is best so the trend doesn't appear lagged. Moving averages are also sometimes used as features. In this case, you would want to take the average only of past values to avoid leaking information backward from the future.

# The Time Dummy #

Because the trend depends only on time, we can model it with a time-step feature. Such a feature is called a **time dummy**, which is essentially an ordinal encoding for a column of timestamps:

| Date    | Time |
|---------|------|
| 1959-06 | 1.0  |
| 1959-07 | 2.0  |
| 1959-08 | 3.0  |
| 1959-09 | 4.0  |
| 1959-11 | 6.0  |
| ...     | ...  |



A time dummy implicitly includes all the timestamps within the range of dates of the time series, including missing values. Generally, this means that if your time series skips a date, your time dummy should skip the corresponding code. Above, the `Date` column skipped `1959-11`, so `Time` needed to skip `5.0`. This only applies to timestamps that are truly missing, however. Many financial time series only include business days, for instance, but we still wouldn't account for weekends or holidays in the trend because we assume nothing happened then.

Trend in a time series can manifest itself in curves with a variety of shapes. Two simple trend patters, linear and quadratic, are displayed here:

<figure style="padding: 1em;">
<img src="https://i.imgur.com/KFYlgGm.png" width=400, alt="Above, Cars Sold in Quebec: an undulating plot gradually increasing from 1960-01 to 1968-12 with a linear trend-line superimposed. Below, Plastics Production in Australia: an undulating plot with a concave-up quadratic trend-line superimposed.">
<figcaption style="textalign: center; font-style: italic"><center><strong>Top:</strong> Series with a linear trend. <strong>Below:</strong> Series with a quadratic trend.
</center></figcaption>
</figure>

A linear model will create a linear trend line when given the time dummy. To create a quadratic trend line, we only need to include an additional column with the time dummy's squared values:

| Date    | Time  (linear) | Time Squared (quadratic) |
|---------|----------------|--------------------------|
| 1959-06 | 1.0            | 1.0                      |
| 1959-07 | 2.0            | 4.0                      |
| 1959-08 | 3.0            | 9.0                      |
| 1959-09 | 4.0            | 16.0                     |
| 1959-11 | 6.0            | 36.0                     |
| ...     | ...            | ...                      |


It's possible to create higher-order polynomial trends by taking higher powers of the time dummy, but polynomials of higher order can diverge rapidly outside the training data making them unreliable when used for forecasting.

Trend can manifest in a variety of ways. The figure illustrates some patterns you might encounter:

<figure style="padding: 1em;">
<img src="https://i.imgur.com/7cxTLCt.png" width=400, alt="">
<figcaption style="textalign: center; font-style: italic"><center>Trend patterns in four time series. <strong>Top left:</strong> Irregular trend. <strong>Top right:</strong> No trend. <strong>Bottom left:</strong> Trend saturating below. <strong>Bottom right:</strong> Trend with changepoints.
</center></figcaption>
</figure>

For fitting more complicated trends, you could consider a *generalized additive modebl* or *GAM* (available in the `statsmodels` package) as an alternative to linear regression.

<blockquote style="margin-right:auto; margin-left:auto; background-color: #ebf9ff; padding: 1em; margin:24px;">
<strong>statsmodels</strong>

<a href="https://www.statsmodels.org/stable/index.html"><code>statsmodels</code></a> is a Python library for statistical models and estimation -- it's like the scikit-learn of classical regression and implements a lot of R's core functionality.

In addition to ordinary regression, `statsmodels` also has a comprehensive set of tools for time series analysis, much of which we'll make use of in this course.
</blockquote>

# Example - Tunnel Traffic #

In this example we'll create a trend model for the *Tunnel Traffic* dataset. Though not required for the exercise, you might like to examine the code in the hidden cell to see how to use Pandas to prepare a time series.

In [ ]:
#$HIDE_INPUT$
from pathlib import Path
from warnings import simplefilter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

simplefilter("ignore")  # ignore warnings to clean up output cells

# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True, figsize=(11, 5))
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    legend=False,
)


# Load Tunnel Traffic dataset
data_dir = Path("../input/ts-course-data")
tunnel = pd.read_csv(data_dir / "tunnel.csv", parse_dates=["Day"])

# Create a time series in Pandas by setting the index to a date
# column. We parsed "Day" as a date type by using `parse_dates` when
# loading the data.
tunnel = tunnel.set_index("Day")

# By default, Pandas creates a `DatetimeIndex` with dtype `Timestamp`
# (equivalent to `np.datetime64`, representing a time series as a
# sequence of measurements taken at single moments. A `PeriodIndex`,
# on the other hand, represents a time series as a sequence of
# quantities accumulated over periods of time. Periods are often
# easier to work with, so that's what we'll use in this course.
tunnel = tunnel.to_period()

Let's make a moving average plot to see what kind of trend this series has. The *Tunnel Traffic* series has an annual seasonality, so we'll use a 365-day window. (You'll learn how to estimate seasonality in the next lesson.)

To create a moving average, first use the `rolling` method to begin a windowed computation. Follow this by the `mean` method to compute the average over the window.

In [ ]:
moving_average = tunnel.rolling(
    window=365,       # 365-day window
    center=True,      # puts the average at the center of the window
    min_periods=183,  # choose about half the window size
).mean()              # compute the mean (could also do median, std, min, max, ...)

ax = tunnel.plot(style=".", color="0.5")
moving_average.plot(
    ax=ax, linewidth=3, title="Tunnel Traffic - 365-Day Moving Average", legend=False,
);

When creating time series features, we can avoid a lot of tricky edge cases by using a library function instead of using Pandas directly. `DeterministicProcess` is a utility from the `statsmodels` library for creating time series features. It accomodates missing timestamps and can generate features for times outside of the training data (much like Prophet's `make_future_dataframe` we saw in Lesson 1). The `constant` argument creates a feature for the y-intercept of the graph, and the `order` argument creates a feature for the trend. "Order" here refers to polynomial order: 1 is linear, 2 is quadratic, 3 is cubic, and so on. The moving average plot suggests that the trend is close to linear, so we'll create a time dummy of order 1.

In [ ]:
from statsmodels.tsa.deterministic import DeterministicProcess

dp = DeterministicProcess(
    index=tunnel.index,  # dates from the training data
    constant=True,       # the level
    order=1,             # the trend
    drop=True,           # drop terms to avoid collinearity
)
X = dp.in_sample()  # features for the training data

X.head()

By the way, a "deterministic process" is a time series that is non-random or completely determined. Our trend and seasonal features are deterministic because they depend only on dates and times, which are fixed. In contrast, a "stochastic process" is a series that is random or unpredictable to some degree. Anything truly requiring a forecast -- like tunnel traffic -- is stochastic. The distinction between deterministic and stochastic processes is fundamental to forecasting; we'll return to it again in Lesson 4.

Now let's create the trend model. As mentioned before, we'll use `LinearRegression` from scikit-learn.

In [ ]:
from sklearn.linear_model import LinearRegression

y = tunnel["NumVehicles"]  # the target

# The intercept is the same as the `const` feature from
# DeterministicProcess. LinearRegression behaves badly with duplicated
# features, so we need to be sure to exclude it here.
model = LinearRegression(fit_intercept=False)
model.fit(X, y)

y_pred = pd.Series(
    model.predict(X),
    index=y.index,
)


ax = tunnel.plot(style=".", color="0.5", title="Tunnel Traffic - Linear Trend")
_ = y_pred.plot(ax=ax, linewidth=3, label="Trend")

The trend discovered by our `LinearRegression` model is almost identical to the moving average plot, which suggests that a linear trend was the right decision in this case.

To make a forecast, we apply our model to "out of sample" features. "Out of sample" refers to times outside of the observation period of the training data, which would include the future times of a forecast. Here's how we could make a 30-day forecast:

In [ ]:
X = dp.out_of_sample(steps=30)

y_fore = pd.Series(
    model.predict(X),
    index=X.index,
)

y_fore.head()

Let's plot a portion of the series to see the trend forecast for the next 30 days:

In [ ]:
ax = tunnel["2005-05":].plot(title="Tunnel Traffic - Linear Trend Forecast", **plot_params)
ax = y_pred["2005-05":].plot(ax=ax, linewidth=3, label="Trend")
ax = y_fore.plot(ax=ax, linewidth=3, label="Trend Forecast", color="C3")
_ = ax.legend()

The trend models we learned about in this lesson turn out to be useful for a number of reasons. Besides acting as a starting point for more sophisticated models, they also make for a useful baseline forecast and as part of a preprocessing step for ML models unable to learn trends (like XGBoost and random forests). We'll learn more about these techniques later in this course.

# Your Turn #
